In [0]:
import os
import json
import gensim
import numpy as np
import nltk
from gensim import corpora, models, similarities
import pickle
os.chdir("drive/My Drive/chatbot")
os.listdir()

In [4]:
import gensim.models.keyedvectors as word2vec
model = word2vec.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
with open("conversation.json") as file:
  data = json.load(file)

In [0]:
corpo = data["conversations"]
inp = []
out = []

for i in range(len(corpo)):
  for j in range(len(corpo[i]) - 1):
    inp.append(corpo[i][j])
    out.append(corpo[i][j+1])
    

In [7]:
import pandas as pd
df = pd.DataFrame({"Input" : inp, "Output" : out})
df.head()


,Input,Output
0,"Good morning, how are you?","I am doing well, how about you?"
1,"I am doing well, how about you?",I'm also good.
2,I'm also good.,That's good to hear.
3,That's good to hear.,Yes it is.
4,Hello,Hi


In [0]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

xtok = [tokenizer.tokenize(x.lower()) for x in inp]
ytok = [tokenizer.tokenize(y.lower()) for y in out]

In [0]:
xvecs = []
for sen in xtok:
  v = [model[w] for w in sen if w in model.vocab]
  xvecs.append(v)

yvecs = []
for sen in ytok:
  v = [model[w] for w in sen if w in model.vocab]
  yvecs.append(v)

In [0]:
eos = np.ones((300), dtype = np.float32)
for tok_sent in xvecs:
  if len(tok_sent) < 15:
    for i in range(15 - len(tok_sent)):
      tok_sent.append(eos)
  else :
    tok_sent[14:] = []
    tok_sent.append(eos)

for tok_sent in yvecs:
  if len(tok_sent) < 15:
    for i in range(15 - len(tok_sent)):
      tok_sent.append(eos)
  else :
    tok_sent[14:] = []
    tok_sent.append(eos)






In [0]:
with open("data.pickle", "wb" ) as f:
  pickle.dump([xvecs, yvecs], f)


In [0]:
# import pickle
# with open("data.pickle", "rb" ) as f:
#   xvecs, yvecs = pickle.load(f)


In [13]:
x_arr = np.array(xvecs, dtype = np.float64)
y_arr = np.array(yvecs, dtype = np.float64)
x_arr.shape



(86, 15, 300)

In [0]:
from keras.models import Model

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten, LSTM, Input, Embedding
from keras.layers import BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras import  backend as K


In [0]:
xtrain, xtest, ytrain, ytest = train_test_split(x_arr, y_arr)

In [16]:
model = Sequential()
model.add(LSTM(output_dim=300,input_shape=xtrain.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=xtrain.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))

model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 15, 300)           721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 300)           721200    
Total params: 1,442,400
Trainable params: 1,442,400
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

model.fit(xtrain, ytrain, nb_epoch=1000, validation_data=(xtest, ytest), callbacks =[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100), ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)])
model.save('model.h5')

In [0]:
from keras.models import load_model
import gensim
import gensim.models.keyedvectors as word2vec
import os
os.chdir("drive/My Drive/chatbot")

bot_model = load_model('model.h5')

w2v = word2vec.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)





In [0]:
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
eos = np.ones((300), dtype = np.float32)
x = input("Enter query")
x = tokenizer.tokenize(x)
vec = [w2v[i] for i in x if i in w2v.vocab]

if len(vec) < 15:
  for i in range(15 - len(vec)):
    vec.append(eos)
else :
  vec[14:] = []
  vec.append(eos)
vec = np.array([vec])
pred = bot_model.predict(vec)

word_list_out = [w2v.similar_by_vector(v)[0] for v in pred[0]]
print(" ".join(word_list_out))



